In [14]:
import os, cv2, re, random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import layers, models, optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
img_width = 150
img_height = 150
TRAIN_DIR = 'train/'
TEST_DIR = 'test/'
train_images_dogs_cats = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] # use this for full dataset
test_images_dogs_cats = [TEST_DIR+i for i in os.listdir(TEST_DIR)]

### Helper functions

In [4]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

### **Sort the traning set. Use 1300 images each of cats and dogs instead of all 25000 to speed up the learning process.**


In [5]:
train_images_dogs_cats.sort(key=natural_keys)
train_images_dogs_cats = train_images_dogs_cats[0:1300] + train_images_dogs_cats[12500:13800] 

test_images_dogs_cats.sort(key=natural_keys)

In [6]:
def prepare_data(list_of_images):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """
    x = [] # images as arrays
    y = [] # labels
    
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (img_width,img_height), interpolation=cv2.INTER_CUBIC))
    
    for i in list_of_images:
        if 'dog' in i:
            y.append(1)
        elif 'cat' in i:
            y.append(0)
        #else:
            #print('neither cat nor dog name present in images')
            
    return x, y

In [7]:
X, Y = prepare_data(train_images_dogs_cats)
print(K.image_data_format())

channels_last


In [8]:
# First split the data in two sets, 80% for training, 20% for Val/Test)
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size=0.2, random_state=1)

In [9]:
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
batch_size = 16

In [10]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0      

In [11]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [12]:
train_generator = train_datagen.flow(np.array(X_train), Y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(np.array(X_val), Y_val, batch_size=batch_size)

In [15]:
with tf.device('/gpu:0'):
    history = model.fit_generator(
        train_generator, 
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=30,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size
    )

Epoch 1/30
130/130 [==============================] - 12s 90ms/step - loss: 0.7084 - acc: 0.5399 - val_loss: 0.6604 - val_acc: 0.5957
Epoch 2/30
130/130 [==============================] - 9s 72ms/step - loss: 0.6611 - acc: 0.6168 - val_loss: 0.6133 - val_acc: 0.6699
Epoch 3/30
130/130 [==============================] - 9s 73ms/step - loss: 0.6283 - acc: 0.6524 - val_loss: 0.6074 - val_acc: 0.6855
Epoch 4/30
130/130 [==============================] - 10s 75ms/step - loss: 0.6036 - acc: 0.6937 - val_loss: 0.5610 - val_acc: 0.7090
Epoch 5/30
130/130 [==============================] - 10s 77ms/step - loss: 0.5828 - acc: 0.7034 - val_loss: 0.5353 - val_acc: 0.7363
Epoch 6/30
130/130 [==============================] - 10s 78ms/step - loss: 0.5669 - acc: 0.7255 - val_loss: 0.5496 - val_acc: 0.7129
Epoch 7/30
130/130 [==============================] - 10s 76ms/step - loss: 0.5522 - acc: 0.7413 - val_loss: 0.5587 - val_acc: 0.7012
Epoch 8/30
130/130 [==============================] - 10s 75ms/s

In [16]:
model.save_weights('model_wieghts.h5')
model.save('model_keras.h5')

In [17]:
X_test, Y_test = prepare_data(test_images_dogs_cats) #Y_test in this case will be []

In [18]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [39]:
test_generator = test_datagen.flow(np.array(X_test), batch_size=batch_size)
prediction_probabilities = model.predict_generator(test_generator, verbose=1, steps=782)

782/782 [==============================] - 9s 11ms/step


In [40]:
prediction_probabilities.shape

(12500, 1)

In [33]:
test_generator

In [30]:
len(X_test)

12500

In [41]:
counter = range(1, len(test_images_dogs_cats) + 1)
solution = pd.DataFrame({"id": counter, "label":list(prediction_probabilities)})
cols = ['label']

for col in cols:
    solution[col] = solution[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)

solution.to_csv("dogsVScats.csv", index = False)